<a href="https://colab.research.google.com/github/1900690/eye-camera/blob/main/eye_camera_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import shutil
name = "選定２"
shutil.copy("/content/drive/MyDrive/budou/"+name+".mp4","/content/"+name+".mp4")
shutil.copy("/content/drive/MyDrive/budou/"+name+".json","/content/"+name+".json")

'/content/選定２.json'

#十字印付け

In [ ]:
from pandas.core.base import doc
import shutil
import os
from google.colab import files
import pandas as pd
from PIL import Image
import cv2

#uploaded = files.upload()
#filename = os.path.splitext(list(uploaded.keys())[0])[0]
filename = "選定２"

#視線データを読み込み必要な部分を抽出
json_file = "/content/"+filename+".json"
df = pd.read_json(json_file)
df2=df["EyeData"].iloc[2]
df3 = pd.json_normalize(df2)
df6=df3[['recomValid','recomPoint.x','recomPoint.y']]

# 動画ファイルのキャプチャー
cap = cv2.VideoCapture("/content/"+filename+".mp4")

# 動画ファイルのフレームレート取得
fps = cap.get(cv2.CAP_PROP_FPS)

#動画のサイズを取得
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width,height)
point_size=max(width,height)

# 保存用動画ファイルのフォーマット設定
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter("/content/"+filename+"_eye_point.mp4", fourcc, fps, size)

# 動画を1コマずつ取り込んで処理
while(cap.isOpened()):
    ret, frame = cap.read() # キャプチャー画像の取り込み

    if ret==True: # キャプチャー画像がある場合
        # 現在時間を取得
        time = cap.get(cv2.CAP_PROP_POS_MSEC)

        # 現在のフレーム数を取得
        frame_n = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # 視点を描画
        cv2.drawMarker(frame, (int(df6.loc[int(frame_n)][1]*width),int(df6.loc[int(frame_n)][2]*height)), (0, 0, 255),cv2.MARKER_CROSS,int(point_size/10),int(point_size/100))
        #print((int(df6.loc[int(frame_n)][1]*width),int(df6.loc[int(frame_n)][2]*height)))
        print(frame_n)

        # mp4動画を保存
        out.write(frame)

    else: # キャプチャー画像がない場合はループ終了
        break

cap.release() # 再生画像をクローズ
out.release() # 出力動画ファイルをクローズ
#files.download("/content/"+filename+"_eye_point.mp4")

#ヒートマップ例

In [11]:
import cv2
import numpy as np
from tqdm import tqdm

def GaussianMask(sizex,sizey, sigma=33, center=None,fix=1):
    """
    sizex  : mask width
    sizey  : mask height
    sigma  : gaussian Sd
    center : gaussian mean
    fix    : gaussian max
    return gaussian mask
    sizex: マスクの幅 (ピクセル数)
    sizey: マスクの高さ (ピクセル数)
    sigma (オプション): ガウス分布の標準偏差 (デフォルト: 33)
    center (オプション): ガウス分布の中心座標 (x, y) のタプル (デフォルト: 画像の中心)
    fix (オプション): マスクの最大値 (デフォルト: 1)
    """
    x = np.arange(0, sizex, 1, float)
    y = np.arange(0, sizey, 1, float)
    x, y = np.meshgrid(x,y)

    if center is None:
        x0 = sizex // 2
        y0 = sizey // 2
    else:
        if np.isnan(center[0])==False and np.isnan(center[1])==False:
            x0 = center[0]
            y0 = center[1]
        else:
            return np.zeros((sizey,sizex))

    return fix*np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / sigma**2)

def Fixpos2Densemap(fix_arr, width, height, imgfile, alpha=0.5, threshold=10):
    """
    fix_arr   : fixation array number of subjects x 3(x,y,fixation)
    width     : output image width
    height    : output image height
    imgfile   : image file (optional)
    alpha     : marge rate imgfile and heatmap (optional)
    threshold : heatmap threshold(0~255)
    return heatmap
    fix_arr: 注視位置データの配列 (x軸とｙ軸,3列目は注視時間)
    width: 出力画像の幅 (ピクセル数)
    height: 出力画像の高さ (ピクセル数)
    imgfile (オプション): 背景画像へのマージング用画像ファイルパス (デフォルト: なし)
    alpha (オプション): 背景画像とヒートマップのマージング率 (デフォルト: 0.5)
    threshold (オプション): ヒートマップのしきい値 (0~255、しきい値以下の画素は0に)
    """

    heatmap = np.zeros((H,W), np.float32)
    for n_subject in tqdm(range(fix_arr.shape[0])):
        heatmap += GaussianMask(W, H, 33, (fix_arr[n_subject,0],fix_arr[n_subject,1]),
                                fix_arr[n_subject,2])

    # Normalization
    heatmap = heatmap/np.amax(heatmap)
    heatmap = heatmap*255
    heatmap = heatmap.astype("uint8")

    if imgfile.any():
        # Resize heatmap to imgfile shape
        h, w, _ = imgfile.shape
        heatmap = cv2.resize(heatmap, (w, h))
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        # Create mask
        mask = np.where(heatmap<=threshold, 1, 0)
        mask = np.reshape(mask, (h, w, 1))
        mask = np.repeat(mask, 3, axis=2)

        # Marge images
        marge = imgfile*mask + heatmap_color*(1-mask)
        marge = marge.astype("uint8")
        marge = cv2.addWeighted(imgfile, 1-alpha, marge,alpha,0)
        return marge

    else:
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        return heatmap

if __name__ == '__main__':
    # Load image file
    img = cv2.imread('/content/images.jpg')

    # Generate toy fixation data
    # ★ここを自分のデータと入れ替える
    num_subjects = 40
    H, W, _ = img.shape

    fix_arr = np.random.randn(num_subjects,3)#標準正規分布に従う乱数を生成
    fix_arr -= fix_arr.min()
    fix_arr /= fix_arr.max()
    fix_arr[:,0] *= W
    fix_arr[:,1] *= H

    # Create heatmap
    heatmap = Fixpos2Densemap(fix_arr, W, H, img, 0.7, 5)
    cv2.imwrite("/content/output.png",heatmap)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [10]:
fix_arr

array([[2.75000000e+02, 6.13208877e+01, 4.06445879e-01],
       [1.22372065e+02, 6.58172350e+01, 5.36906548e-01],
       [2.03653620e+02, 5.78360706e+01, 6.56212674e-01],
       [9.72611027e+01, 5.16853474e+01, 4.02911293e-01],
       [1.71382945e+02, 1.53797331e+02, 3.31143878e-01],
       [9.44424179e+01, 7.45653612e+01, 4.72217747e-01],
       [8.75821559e+01, 9.13304346e+01, 3.41051378e-01],
       [1.55432025e+02, 6.78175887e+01, 4.35131359e-01],
       [1.11304881e+02, 6.91457403e+01, 4.26751483e-01],
       [1.37596951e+02, 9.06829446e+01, 1.71126211e-01],
       [9.71119522e+01, 6.44238564e+01, 2.05711753e-01],
       [1.52805507e+02, 6.71867151e+01, 8.04432392e-01],
       [1.15860855e+02, 9.43069792e+01, 6.46284231e-01],
       [1.40267947e+02, 5.55315315e+01, 5.40315785e-01],
       [1.58389431e+02, 1.25945302e+02, 7.21420837e-01],
       [1.63937556e+02, 4.88613269e+01, 6.64080386e-01],
       [1.03451982e+02, 8.88537271e+01, 4.24012000e-01],
       [1.79502249e+02, 1.16350